In [1]:
# step 20
import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    return Mul()(x0, x1)


Variable.__add__ = add
Variable.__mul__ = mul

a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

# y = add(mul(a, b), c)
y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


- 연산자는 좌항의 인스턴스에 대한 특수 (연산자) 메서드부터 호출해서 연산한다.
- 좌항의 인스턴스에 특수 메서드가 존재하지 않으면 우항의 인스턴스에 대한 특수 연산자를 호출하여 연산한다.

ex ) a * b 의 연산에서 $__mul__(self, other)$에서 인수가 전달되는 방식  
```
self <- a  

other <- b  
```
$*$ 덧셈도 마찬가지

# step 21
연산자 오버로드(2)  
아직 안되는것
- ndarray 인스턴스와의 계산  
  ex) a * np.array(2.0)
- 수치 데이터(int, float 등)와의 계산  
  ex) 3 + b


ndarray 인스턴스와 계산할 수 있게 하기

In [2]:
# ndarray 인스턴스와 계산할 수 있게 하기 1
def as_variable(obj):
  '''
  # 인수로 주어진 객체를 Variable 인스턴스로 변환해주는 함수
    ndarray 인스턴스와 Variable 인스턴스를 함께 사용하기 위해 ndarray 인스턴스를 Variable 인스턴스로 변환한다.
  '''
  if isinstance(obj, Variable):
    return obj
  return Variable(obj)

In [3]:
# # ndarray 인스턴스와 계산할 수 있게 하기 2
class Function:
  def __call__(self, *inputs):
    # 처음부터 as_variable 메서드를 거치게 해서 모든 인수를 Variable 인스턴스 형태로 만듬
    inputs = [as_variable(x) for x in inputs]

    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])
      for output in outputs:
          output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]

    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
      y = x0 + x1
      return y

    def backward(self, gy):
      return gy, gy


def add(x0, x1):
  return Add()(x0, x1)


class Mul(Function):
  def forward(self, x0, x1):
    y = x0 * x1
    return y

  def backward(self, gy):
    x0, x1 = self.inputs[0].data, self.inputs[1].data
    return gy * x1, gy * x0


def mul(x0, x1):
  return Mul()(x0, x1)

In [4]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

# 제대로 작동해서 결과값(y)이 variable 인스턴스로

variable(5.0)


수치 데이터(int, float 등)와 계산 할 수 있게 하기  
- int
- float
- np.float64, np.int64 같은 타입도 함께 사용할 수 있도록 변경

In [5]:
# 수치 데이터(int, float 등)와 계산 할 수 있게 하기 1
class Function:
  def __call__(self, *inputs):
    inputs = [as_variable(x) for x in inputs]
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])
      for output in outputs:
          output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]

    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
      y = x0 + x1
      return y

    def backward(self, gy):
      return gy, gy

# 기존의 코드
# def add(x0, x1):
#   return Add()(x0, x1)

# x + 3.0 같은 코드를 실행 할 수 있게 할려면,
def add(x0, x1):
  # 3.0 부분인 x1을 as_array 메서드로 형태를 바꿔주면 된다.
  x1 = as_array(x1)
  return Add()(x0, x1)


class Mul(Function):
  def forward(self, x0, x1):
    y = x0 * x1
    return y

  def backward(self, gy):
    x0, x1 = self.inputs[0].data, self.inputs[1].data
    return gy * x1, gy * x0


def mul(x0, x1):
  return Mul()(x0, x1)

Variable.__add__ = add
Variable.__mul__ = mul

In [6]:
x = Variable(np.array(2.0))
y = x + 3.0
print(y)

# 이후에 Function 클래스에서 Variable 인스턴스로 변환되기 때문에 ndarray 인스턴스로 바꿨다고 해서 걱정할 필요는 없다.

variable(5.0)


하지만 지금까지 코드로는 x + 3.0나 x + np.array(2.0) 처럼 **좌항은 Variable 인스턴스이고, 우항이 Variable 인스턴스가 아닌 경우에만 할 수 있다**는 문제가 생긴다.  
=> **자리가 바뀌면 계산을 못함!!**


먼저 수치데이터가 좌항일 때 계산을 하지 못하는 이유 : 수치 데이터에는 해당 연산자의 특수 메서드가 없어서 호출할게 없기 때문에  

그러면 수치 데이터가 어느쪽에 있든 계산할 수 있도록 변경할 필요가 있다.  
ex 1) x + 3.0 의 경우  
연산자의 좌항에 있는 x의 특수 메서드($__add__$)를 호출하게 한다.  
ex 1) 3.0 + x 의 경우
연산자의 우항에 있는 x의 특수 메서드($__radd__$)를 호출하게 한다.  
$*$$__radd__$의 r은 right 

ex ) 2.0 * b 의 연산에서 $__rmul__(self, other)$에서 인수가 전달되는 방식  
```
self <- b  

other <- 2.0  
```
$*$ 덧셈도 마찬가지

```
Variable.__radd__ = add
Variable.__rmul__ = mul
```
+, * 같은 연산자로도 사용할 수 있게 파이썬 함수로도 구현한다.

In [7]:
class Function:
  def __call__(self, *inputs):
    inputs = [as_variable(x) for x in inputs]
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys, tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])
      for output in outputs:
          output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]

    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self, xs):
    raise NotImplementedError()

  def backward(self, gys):
    raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
      y = x0 + x1
      return y

    def backward(self, gy):
      return gy, gy

def add(x0, x1):
  x1 = as_array(x1)
  return Add()(x0, x1)


class Mul(Function):
  def forward(self, x0, x1):
    y = x0 * x1
    return y

  def backward(self, gy):
    x0, x1 = self.inputs[0].data, self.inputs[1].data
    return gy * x1, gy * x0


def mul(x0, x1):
  x1 = as_array(x1)
  return Mul()(x0, x1)


# 연산자의 좌항에 있을 경우
Variable.__add__ = add
Variable.__mul__ = mul

# 연산자의 우항에 있을 경우
Variable.__radd__ = add
Variable.__rmul__ = mul

In [8]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

y = x + 3.0
print(y)

y = 3.0 * x + 1.0
print(y)

variable(5.0)
variable(5.0)
variable(7.0)


그러면 좌항이 ndarray 인스턴스 일 때는?  
ndarray 인스턴스 안에는 연산을 해주는 특수 메서드가 지정되어있다.  
그래서 ndarray 인스턴스가 좌항에 올 경우 연산자는 ndarray 인스턴스에서 호출한 메소드로 연산하게 된다.  

하지만 여기서는 ndarray 인스턴스가 아닌 Variable 인스턴스 일 때의 연산과정이 필요하기 때문에 좌항에 ndarray 인스턴스가 있더라도 Variable 인스턴스의 메소드를 먼저 호출 할 수 있도록 해야한다.  
=> 연산자 우선순위를 지정하자!  

Variable 인스턴스의 속성에 $__array_priority__$를 추가하고 그 값을 큰 정수로 설정하면 된다.  

$__array_priority__$에 대한 더 자세한 설명을 보고싶으면 아래의 링크를 참조할 것.  
[Standard array subclasses](https://numpy.org/doc/stable/reference/arrays.classes.html#numpy.class.__array_ufunc__)

In [9]:
class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    # __array_priority__의 기본값은 0.0
    # 값이 클 수록 우선순위가 높아진다.
    # ndarray 인스턴스의 속성중 하나 
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)


Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

In [10]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

y = np.array(3.0) + x
print(y)

variable(5.0)
variable(5.0)


# step 22

In [11]:
# 절대값을 취해주는 것과 같은 부호 변환 연산자
# 단항 연산자라서 들어오는 인수도 하나기 때문에 좌항우항 위치 상관 안해도 됨
class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy


def neg(x):
    return Neg()(x)

In [12]:
# 뺄셈
# 덧셈과 곱셈은 좌항과 우항의 순서가 바껴도 계산결과 같기 때문에 __add__와 __radd__처럼 지정은 해주지만 계산 자체에 구별할 필요가 없다. (둘다 결국 add 메서드로 계산되기 때문에)
# 뺄셈은 계산에 구별(좌항과 우항의 위치변화에 따라)이 필요하기 때문에 직접 rsub 메서드를 지정해 준다. 
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy

# x0 - x1
def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

# x1 - x0
def rsub(x0, x1):
    x1 = as_array(x1)
    return sub(x1, x0)

In [13]:
# 나눗셈
# 뺄셈과 마찬가지로 인수의 위치구별이 필요하기 때문에 rdiv 메서드를 지정해준다.
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1


def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)


def rdiv(x0, x1):
    x1 = as_array(x1)
    return div(x1, x0)

In [14]:
# 거듭제곱
# x를 밑, c를 지수로 할 경우
# 실전에서는 거의 사용되지 않으므로 연습정도로 작성함
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c

        gx = c * x ** (c - 1) * gy
        return gx


def pow(x, c):
    return Pow(c)(x)

```
Variable.__neg__ = neg
Variable.__sub__ = sub
Variable.__rsub__ = rsub
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv
Variable.__pow__ = pow
```
$-$, $**$, $/$으로도 사용 가능하게 파이썬 함수로도 구현한다.

In [15]:
import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)


class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy


def neg(x):
    return Neg()(x)


class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy


def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)


def rsub(x0, x1):
    x1 = as_array(x1)
    return sub(x1, x0)


class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1


def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)


def rdiv(x0, x1):
    x1 = as_array(x1)
    return div(x1, x0)


class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c

        gx = c * x ** (c - 1) * gy
        return gx


def pow(x, c):
    return Pow(c)(x)


Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul
Variable.__neg__ = neg
Variable.__sub__ = sub
Variable.__rsub__ = rsub
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv
Variable.__pow__ = pow

In [17]:
x = Variable(np.array(2.0))
y = -x
print('-x :', y)  # variable(-2.0)

y1 = 2.0 - x
y2 = x - 1.0
print('2.0 - x =', y1)  # variable(0.0)
print('x - 1.0 =', y2)  # variable(1.0)

y = 3.0 / x
print('3.0 / x = ', y)  # variable(1.5)

y = x ** 3
y.backward()
print('x ** 3 = ', y)  # variable(8.0)

-x : variable(-2.0)
2.0 - x = variable(0.0)
x - 1.0 = variable(1.0)
3.0 / x =  variable(1.5)
x ** 3 =  variable(8.0)
